#### 设置样式显示效果

In [15]:

'''
设置悬停效果
'''
def hover(hover_color="#ffff99"):
    return dict(selector="tr:hover",
                props=[("background-color", "%s" % hover_color)])
'''
美化DataFrame显示效果
'''
def display_prettify(df):
    from IPython.display import HTML

    styles = [
        hover(),
        dict(selector="th", props=[("font-size", "100%"),
                                   ("text-align", "center")]),
        dict(selector="td", props=[("text-align", "left")]),
        dict(selector="caption", props=[("caption-side", "left")])
    ]
    return df.style.set_table_styles(styles).set_caption("Hover to highlight.")

###  取得 MariaDB 版本

In [18]:
ip,user,pwd,dnname,port='192.168.182.131','root','root','mysql',3306

import MySQLdb as mdb

conn = None
try:
    '''
    1 connect mysql
        type : MySQLdb.connections.Connection
        func : connect(ip,user,password,dbname)
    '''
    conn = mdb.connect(ip,user,pwd,dnname,port)
    cursor = conn.cursor()
    
    '''
    2 fetchone
       单条结果，返回单个元组 
    '''
    cursor.execute("SELECT VERSION();")  # 执行查询
    row = cursor.fetchone()
    
    '''
    3 fetchall
       所有结果，返回二维元组 
    '''
    cursor.execute("select * from user;")  # 执行查询
    rows = cursor.fetchall()
finally:
    if conn:
        conn.close() # 关闭连接
        cursor.close()

In [20]:
import pandas as pd

rw_df=pd.DataFrame([t for t in rows])
display_prettify(rw_df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45
0,localhost,root,*81F5E21E35407D884A6CD4A731AEBFB6AF209E1B,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,,,,,0,0,0,0,,*81F5E21E35407D884A6CD4A731AEBFB6AF209E1B,N,N,,0.000000
1,127.0.0.1,root,*81F5E21E35407D884A6CD4A731AEBFB6AF209E1B,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,,,,,0,0,0,0,,*81F5E21E35407D884A6CD4A731AEBFB6AF209E1B,N,N,,0.000000
2,::1,root,*81F5E21E35407D884A6CD4A731AEBFB6AF209E1B,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,,,,,0,0,0,0,,,N,N,,0.000000
3,%,root,*81F5E21E35407D884A6CD4A731AEBFB6AF209E1B,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,,,,,0,0,0,0,,,N,N,,0.000000
4,localhost,root1,*81F5E21E35407D884A6CD4A731AEBFB6AF209E1B,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,N,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,,,,,0,0,0,0,,,N,N,,0.000000
5,hostname,username,*81F5E21E35407D884A6CD4A731AEBFB6AF209E1B,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,,,,,0,0,0,0,,,N,N,,0.000000
6,192.168.%,root,*81F5E21E35407D884A6CD4A731AEBFB6AF209E1B,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,N,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,,,,,0,0,0,0,,,N,N,,0.000000


### 创建一个表并且插入数据

In [27]:
import MySQLdb as mdb

# conn 创建全局连接
conn = mdb.connect(ip,user,pwd,dnname,port);
with conn:
    curs = conn.cursor() # 获取 cursor
    # 创建数据表 writers(id,name)
    curs.execute("CREATE TABLE IF NOT EXISTS \
        Writers(Id INT PRIMARY KEY AUTO_INCREMENT, Name VARCHAR(25))")
    # 插入 5 条数据
    curs.execute("INSERT INTO Writers(Name) VALUES('Jack London')")
    curs.execute("INSERT INTO Writers(Name) VALUES('Honore de Balzac')")
    curs.execute("INSERT INTO Writers(Name) VALUES('Lion Feuchtwanger')")
    curs.execute("INSERT INTO Writers(Name) VALUES('Emile Zola')")
    curs.execute("INSERT INTO Writers(Name) VALUES('Truman Capote')")
    
    # conn.commit()
    
curs.execute('SELECT * FROM Writers;')
rows=curs.fetchall()
conn.close()
curs.close()

In [28]:
display_prettify(pd.DataFrame([rw for rw in rows]))

,0,1
0,1,Jack London
1,2,Honore de Balzac
2,3,Lion Feuchtwanger
3,4,Emile Zola
4,5,Truman Capote


### python 获取表数据并取行数据中单个数据

In [53]:
import MySQLdb as mdb

# conn 创建全局连接
conn = mdb.connect(ip,user,pwd,dnname,port);

with conn:
    curs = conn.cursor() # 获取 cursor
    curs.execute("SELECT * FROM Writers")
    
    numrows = int(curs.rowcount) # 获取行数
    for i in range(numrows): # 循环取行数据
        row = curs.fetchone() # 每次取出一行数据，同时指针下移
        print row[0], row[1]
        
    print ('-----使用字典游标获取结果集，并通过字段名访问值-----')
    curs = conn.cursor(mdb.cursors.DictCursor)
    curs.execute("SELECT * FROM Writers")
    rows3=curs.fetchall()
    for row in rows3:
        print "%s %s" % (row["Id"], row["Name"])

conn.close()
curs.close()

1 Jack London
2 Honore de Balzac
3 Guy de Maupasant
4 Guy de Maupasant
5 Truman Capote
-----使用字典游标获取结果集，并通过字段名访问值-----
1 Jack London
2 Honore de Balzac
3 Guy de Maupasant
4 Guy de Maupasant
5 Truman Capote


### Prepared Statements 查询（更安全方便）

In [56]:
import MySQLdb as mdb

conn = mdb.connect(ip,user,pwd,dnname,port);
with conn:
    curs = conn.cursor()
    # 通过组装sql 进行查询操作
    curs.execute("UPDATE Writers SET Name = %s WHERE Id = %s",("Lion Feuchtwanger", "3"))
    # 获取影响多少行
    print "Number of rows updated: %d" % curs.rowcount

conn.commit() 
conn.close()
curs.close()

Number of rows updated: 1


### 把图片用二进制存入 MySQL
- BLOB (Binary Large Object)，即较大的二进制对象字段
- escape_string 
    - 字符转义函数
- CREATE TABLE Images(Id INT PRIMARY KEY AUTO_INCREMENT, Data MEDIUMBLOB);

In [64]:
import MySQLdb as mdb
import sys

# conn 创建全局连接
conn = mdb.connect(ip,user,pwd,dnname,port);
with conn:
    curs = conn.cursor() # 获取 cursor
    # 创建数据表 writers(id,name)
    curs.execute("CREATE TABLE if not exists \
        Images(Id INT PRIMARY KEY AUTO_INCREMENT, Data MEDIUMBLOB);")
curs.close()

try:
    fin = open("/home/synway/kngines1.jpg")
    img = fin.read() # 读入文件流
    fin.close()
except IOError, e:
    print "Error %d: %s" % (e.args[0],e.args[1])
    sys.exit(1)
try:
    cursor = conn.cursor()
    cursor.execute("INSERT INTO Images SET Data='%s'" % mdb.escape_string(img))
    conn.commit()  # 提交数据
    cursor.close()
    conn.close()
except mdb.Error, e:
    print "Error %d: %s" % (e.args[0],e.args[1])
    sys.exit(1)

/root/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: Warning: Table 'Images' already exists


### 从数据库中把图片读出来

In [65]:
import MySQLdb as mdb
import sys
try:
    conn = mdb.connect(ip,user,pwd,dnname,port);
    cursor = conn.cursor() 
    cursor.execute("SELECT Data FROM Images LIMIT 1") # 执行查询
    
    fout = open('image.png','wb')  # 使用二进制进行写文件
    fout.write(cursor.fetchone()[0])
    
    fout.close() # 关闭输出流
    cursor.close() # 关闭游标
    conn.close() # 关闭连接
except IOError, e:
    print "Error %d: %s" % (e.args[0],e.args[1])
    sys.exit(1)

### Transaction，即事务（手动提交，自动回滚）

#### 查询默认引擎信息
- MyISAM
    - 不支持事务；
    - 内部的复杂机制很少，特别适应于读多写少的应用；
- InnoDB
    - 事务型存储引擎，适合处理大量的短期事务；

In [74]:
import MySQLdb as mdb
import sys

conn = mdb.connect(ip,user,pwd,dnname,port);
cursor = conn.cursor()
cursor.execute('show engines;')
rows=cursor.fetchall()

conn.close()
cursor.close()

engines_df=pd.DataFrame([rw for rw in rows], columns=['Engine_name','Support','Comment','Transactions','XA','Savepoints'])
display_prettify(engines_df)

,Engine_name,Support,Comment,Transactions,XA,Savepoints
0,MRG_MyISAM,YES,Collection of identical MyISAM tables,NO,NO,NO
1,CSV,YES,CSV storage engine,NO,NO,NO
2,Aria,YES,Crash-safe tables with MyISAM heritage,NO,NO,NO
3,MyISAM,YES,MyISAM storage engine,NO,NO,NO
4,MEMORY,YES,"Hash based, stored in memory, useful for temporary tables",NO,NO,NO
5,InnoDB,DEFAULT,"Supports transactions, row-level locking, foreign keys and encryption for tables",YES,YES,YES
6,SEQUENCE,YES,Generated tables filled with sequential values,YES,NO,YES
7,PERFORMANCE_SCHEMA,YES,Performance Schema,NO,NO,NO


In [79]:
import MySQLdb as mdb
import sys
try:
    conn = mdb.connect(ip,user,pwd,dnname,port);
    cursor = conn.cursor()
    
    # 如果某个数据库支持事务，会自动开启
    # MariDB，自动开启事务（若是 MyISAM 引擎则不会）
    cursor.execute("UPDATE Writers SET Name = %s WHERE Id = %s",("Leo Tolstoy", "1"))
    cursor.execute("UPDATE Writers SET Name = %s WHERE Id = %s",("Boris Pasternak", "2"))
    cursor.execute("UPDATE Writer SET Name = %s WHERE Id = %s",("Leonid Leonov", "3"))
    
    # 事务的特性 1、原子性的手动提交
    conn.commit()
    cursor.close()
    conn.close()
except mdb.Error, e:
    # 若出现错误，则回滚，即上面三条语句要么执行，要么不执行
    conn.rollback()
    print "Error %d: %s" % (e.args[0],e.args[1])

<font color='gray'>注:<br />
（1）因为不存在 writer 表（SQL 第三条语句），所以出现错误：Error 1146: Table ‘test.writer’ doesn’t exist <br />
（2）出现错误，触发异常处理， 3 条语句的前两条会自动变成没有执行，结果不变.<br />
（3）如果本代码放到一个 MyISAM 引擎表，前两句会执行，第三句不会；如果是 INNDB 引
擎，则都不会执行。（事务特性）</font>